In [36]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pylsd.lsd import lsd
%matplotlib inline

In [18]:
img_path = "/Users/Chihiro/Personal/10_Projects/GPC/get_plant_plate_txt/image/20200411_1.jpg"
output_dir = "/Users/Chihiro/Desktop/"

In [173]:
def get_lines(base_img, img):
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.GaussianBlur(img,(5,5),1)
    cv2.imwrite(output_dir + "img_gaussian.png", img_gray)
    lower = 10
    upper = 250
    img_gray = cv2.Canny(img_gray,lower,upper,apertureSize = 3)

    #直線検出（ハフ変換)
    lines = cv2.HoughLines(img_gray, 
                        rho=1, 
                        theta=np.pi/360, 
                        threshold=1000)

     # 直線検出（確率的ハフ変換）
    lines = cv2.HoughLinesP(img_gray, 
                                                rho=1, 
                                                theta=np.pi/360, 
                                                threshold=20, 
                                                minLineLength=50, 
                                                maxLineGap=5)

    #直線検出（LSD）
    lines = lsd(img_gray)
    
    
    for line in linesH:
        x1, y1, x2, y2 = line[0]

        # 赤線を引く
        img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 10)

In [150]:
def mask_img(img):
    masked_img = np.full_like(img, 255,dtype=np.uint8)
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    for i_row, row in enumerate(img_HSV):
            for i_column, column in enumerate(row):
                    H = column[0] * 2
                    if (165 <= H) and (H <= 200):
                        masked_img[i_row][i_column] = np.array([0, 0, 0],dtype=np.uint8)
                    else:
                        continue

    return masked_img

In [106]:
def binalize(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     img_gray = cv2.GaussianBlur(img_gray,(5,5),0)
    
    
    #  二値化 
#     low_thr = 200
#     hight_thr = 255
#     retval, img_bin = cv2.threshold(img_gray,low_thr,hight_thr,cv2.THRESH_BINARY)
    
    # 大津の二値化 
    low_thr = 0
    hight_thr = 255
    retval, img_bin = cv2.threshold(img_gray, low_thr, hight_thr, cv2.THRESH_OTSU)

#     適応的閾値処理による二値化
#     img_bin = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 39,2)
    
    return img_bin

In [165]:
def get_contours(org_img, img):
    
#     for i in range(2):
#         img, contours, hierarchy = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
#         print(len(contours))
#         cv2.drawContours(org_img, contours, -1, (0,255,0), 3)
    
#     輪郭の近似もやる？
#     epsilon = 0.1*cv2.arcLength(cnt,True)
#     approx = cv2.approxPolyDP(cnt,epsilon,True)
    
    for i in range(1):
        img, contours, hierarchy = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        print(len(contours))

        for i in range(0, len(contours)):
            if len(contours[i]) > 0:        
                # remove small objects
                if cv2.contourArea(contours[i]) < 500:
                    continue

            cv2.polylines(img, contours[i], True, (255, 255, 255), 5)
            
    return img

In [100]:
def noise_reduction(img):
#     kernel = np.ones((10,10)) # カーネルサイズの調整が必要そう
#     img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.medianBlur(img,15)
    cv2.imwrite(f"{output_dir}noise_reducted_img.png",img)
    return img

In [151]:
org_img = cv2.imread(img_path)
img = base_img

img_masked = mask_img(img)

In [170]:
img = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY)
img = noise_reduction(img)

In [171]:
get_contours(org_img,img)

163


array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255]], dtype=uint8)